# Fits

In [1]:
import numpy as np
from lmfit import Model

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.stats import norm

from symfit import variables, parameters, Fit
import symfit as sm

## Functions
- For the background, the following parametric function will be used
$$exp \left( 2 \cdot \frac{|x - a+ib|^2}{|x - c+id|^2} + f \cdot (x - c) \right)$$
- For the signal, we will use either the Gaussian function
$$\mu \cdot exp \left( -\frac{1}{2} \frac{(x - x_0)^2}{\sigma^2} \right)$$
or a Maxwell-Boltzmann distribution
$$\mu \cdot \frac{x^2}{\sigma^3} \cdot exp \left( -\frac{1}{2} \frac{(x - x_0)^2}{\sigma^2} \right)$$
depending on the theoretical model we assume.

The total shape will be given by the sum of the two functions.

In [2]:
def bkg(x, a, b, c, d, e, f):                                                   
    return e**2*abs(x-a+1j*b)**2/abs(x-c+1j*d)**2+f*(x-c)

def gaussian(x, x0, s, mu):
    return mu * np.exp(-.5*((x-x0)/s)**2)
    
def maxwell(x, x0, s, mu):
    return mu * x**2/s**3 * np.exp(-.5*((x-x0)/s)**2)
    
    
def signal_gauss(x, a, b, c, d, e, f,
                    x0, s, mu):
    return bkg(x,a,b,c,d,e,f) + gaussian(x,x0,s,mu)
    
def signal_maxwell(x, a, b, c, d, e, f,
                      x0, s, mu):
    return bkg(x,a,b,c,d,e,f) + maxwell(x,x0,s,mu)

## Background

Parameters $a$ and $c$ need to be initialized to the cavity frequency.

In [3]:
def fit_bkg(x, y, w, center, ref):
    # set fit model
    bkg_model = Model(bkg)
    ps = bkg_model.make_params(a={'value':center, 'min':center*0.999, 'max':center*1.01},
                               b=2e4,
                               c={'value':center, 'min':center*0.999, 'max':center*1.01},
                               d=2.2e4,
                               e=1e-2*np.sqrt(ref),
                               f=1e-12*ref)
    # execute fit
    result = bkg_model.fit(y, x=x, params=ps, weights=1/w)
    
    return result

## Signal + Background

Background parameters will be initialized with the results of a previous fit, and will remain constant to facilitate convergence of the signal part. The $\sigma$ is set constant to 16 bins (default to 10.416 kHz), while the $x_0$ will be made to vary over the whole range of probed frequencies.

In [4]:
def fit_sig(x, y, w, x_0, init_params, signal, mu_init=1, mu_vary=True, par_vary=False):
    
    # set fit model
    sig_model = Model(signal)
    ps = sig_model.make_params(a ={'value':init_params['a'], 'vary':par_vary},
                               b ={'value':init_params['b'], 'vary':par_vary},
                               c ={'value':init_params['c'], 'vary':par_vary},
                               d ={'value':init_params['d'], 'vary':par_vary},
                               e ={'value':init_params['e'], 'vary':par_vary},
                               f ={'value':init_params['f'], 'vary':par_vary},
                               mu={'value':mu_init, 'min':0, 'vary':mu_vary},
                               x0={'value':x_0, 'vary':False},
                               s ={'value':16*651, 'vary':False}) # fixed value to 16 bins

    result = sig_model.fit(y, x=x, params=ps, weights=1/w)
    
    return result

## Plot Results

The following plots will be shown:
- Power vs. frequency, including the best fit function
- Residuals vs. frequency
- Distribution of normalized residual  $\frac{y_{fit} - y}{\sigma}$; ideally, it should be a Gaussian with $\mu = 0$ and $\sigma = 1$
- Residual vs. frequency, highlighting the $\pm \sigma$ belt

In [5]:
def plot_fit(x, y, w, fit_result):
    # prepare canvas
    fig = plt.figure(figsize=(15,10))
    gs  = GridSpec(2, 2)
    ax  = fig.add_subplot(gs[0,:])
    ax1 = fig.add_subplot(gs[1,0])
    ax2 = fig.add_subplot(gs[1,1])
    
    # plot data and best fit
    ax.plot(x, y, 'o', label='data')
    ax.plot(x, fit_result.best_fit, color='red', label='fit')
    
    fmin, fmax = min(x), max(x)
    ax.set_xlim([fmin,fmax])
    ax.legend()
    ax.set_xlabel('Frequence [Hz]')
    ax.set_ylabel('FFT')
    
    #residuals w.r.t. freq
    fit_result.plot_residuals(ax=ax1)
    
    
    # plot histogrm of residuals (with the fit)
    rangeMax = int(np.max(fit_result.residual))+1
    
    ax2.hist(fit_result.residual, bins=15, density=True, range=(-rangeMax,rangeMax))
    
    
    fit_res = norm.fit(fit_result.residual, loc=0, scale=1)
    ax2.plot(np.linspace(-rangeMax, rangeMax, 100),
             norm.pdf(np.linspace(-rangeMax, rangeMax, 100), fit_res[0], fit_res[1]), color="red")
    ax2.axvline(fit_res[0], color='black', linestyle='dashed', linewidth=1)
    
    summary_text = "mean: {}\n std: {}".format(np.round(fit_res[0],3), np.round(fit_res[1], 3))
    ax2.text(0.9, 0.9, summary_text, transform=fig.gca().transAxes, ha='right', va='top')
    ax2.set_xlabel('Residuals')


    # plot of residuals vs. freq and error band
    fig3, ax3 = plt.subplots(1, 1, figsize=(18,5))
    
    ax3.scatter(x, fit_result.residual*w, label="residuals")
    ax3.plot(x,  w, label="+$\sigma$")
    ax3.plot(x, -w, label="-$\sigma$")
    ax3.set_xlim([fmin,fmax])
    ax3.set_xlabel('Frequence [Hz]')
    ax3.set_ylabel('Residual')
    ax3.legend()
    
    plt.show()
    return fit_result

## Fit Multiple Runs Simultaneously

In [6]:
def multipleFitBKG(InfoDataset):
    
    fitResult = []
    for run in InfoDataset:
        # fit background for each individual run separately 
        bkg_result = fit_bkg(run["freq"], run["fft"],
                             run["weights"], run["center"], run["ref"])
            
        results = {"run":run["name"],
                   "background_bestParams":bkg_result.params.valuesdict(),
                   "background_bestfit"   :bkg_result.best_fit,
                   "background_residuals" :bkg_result.residual}
            
        fitResult.append(results)
    
    return(fitResult)

In [7]:
def multipleFitSIG(InfoDataset, fitBkg, x0_, mu_=1, sigma_=16*651):
    
    #variables
    xs = variables(', '.join('x_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    ys = variables(', '.join('y_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    
    #specific parameters
    ap = parameters(', '.join('a_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    bp = parameters(', '.join('b_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    cp = parameters(', '.join('c_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    dp = parameters(', '.join('d_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    ep = parameters(', '.join('e_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    fp = parameters(', '.join('f_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    
    for i in range(len(fitBkg)):
        ap[i].fixed = fitBkg[i]["background_bestParams"]["a"]
        bp[i].fixed = fitBkg[i]["background_bestParams"]["b"]
        cp[i].fixed = fitBkg[i]["background_bestParams"]["c"]
        dp[i].fixed = fitBkg[i]["background_bestParams"]["d"]
        ep[i].fixed = fitBkg[i]["background_bestParams"]["e"]
        fp[i].fixed = fitBkg[i]["background_bestParams"]["f"]
    
    #common parameters
    x0, mu, sigma = parameters("x0, mu, sigma")
    
    x0.fixed = x0_
    sigma.fixed = sigma_
    mu.value = mu_
    
    model_dict = {
        y: e**2*sm.Abs(x-a+1*sm.I*b)**2/sm.Abs(x-c+1*sm.I*d)**2+f*(x-c) + 
           mu*(1/(sm.sqrt(2*sm.pi)*sigma)) * sm.exp(-.5*((x-x0)/sigma)**2)
            
        for x, y, a, b, c, d, e, f in zip(xs, ys, ap, bp, cp, dp, ep, fp)
    }
    
    x = np.empty((len(fitBkg), 200))
    y = np.empty((len(fitBkg), 200))
    
    i = 0
    for run in InfoDataset:
        x[i] = run["freq"]
        y[i] = run["fft"]
        i += 1
            
    fit = Fit(model_dict, 
                    x_1  = x[0],  y_1  = y[0],
                    x_2  = x[1],  y_2  = y[1],
                    x_3  = x[2],  y_3  = y[2],
                    x_4  = x[3],  y_4  = y[3],
                    x_5  = x[4],  y_5  = y[4],
                    x_6  = x[5],  y_6  = y[5],
                    x_7  = x[6],  y_7  = y[6],
                    x_8  = x[7],  y_8  = y[7],
                    x_9  = x[8],  y_9  = y[8],
                    x_10 = x[9],  y_10 = y[9],
                    x_11 = x[10], y_11 = y[10],
                    x_12 = x[11], y_12 = y[11],
                    x_13 = x[12], y_13 = y[12]
    )
    
    fit_result = fit.execute()
    
    #results
    mu_hat = fit_result.value(mu)

    yfit = np.empty((len(fitBkg),200))
    for i in range(len(fitBkg)):
        yfit[i] = signal_gauss(x[i],
                               fit_result.value(ap[i]),
                               fit_result.value(bp[i]),
                               fit_result.value(cp[i]),
                               fit_result.value(dp[i]),
                               fit_result.value(ep[i]),
                               fit_result.value(fp[i]),
                               fit_result.value(x0),
                               fit_result.value(sigma),
                               fit_result.value(mu))
                                  
    i=0
    resultsFit={}
    for run in InfoDataset:
        bestFit=pd.DataFrame({"freq":x[i],"fft":y[i]})
        fitsRes={"run":run["name"],"bestFits":bestFit}
        resultsFit.append(fitsRes)
        i+=1
    
    
    return(resultsFit, mu_hat)